<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-02 13:45:05
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -50.16 K (-0.37%)
Current PnL: -14.04 L (-9.78%)
CY Booked + Current PnL: -7.30 L (-5.09%)
-------------------
Total profit:  4.35 L
Total loss:  -18.39 L
-------------------
Total Booked + Current PnL: 19.68 L (16.89%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.96%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.47 L (53.34%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.44%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,92.71,134136.0,-8343.0,8960.0,0.93,-5.86,6.68,0.44,243.0,-0.93,0.99,58.32,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC
39,HINDALCO,651.95,761.55,-19.08,H-LC,87.50,111256.0,6944.0,10592.0,0.15,6.66,9.52,16.81,27.0,0.66,0.82,23.77,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,-1.39,M-LC,36.46,173154.0,32314.0,12173.0,-1.52,22.94,7.03,31.59,89.0,2.65,1.28,50.41,XY24,NTT,BANKS
5,APOLLOHOSP,6904.43,8285.00,-18.07,H-LC,80.21,111675.0,8109.0,12597.0,-0.68,7.83,11.28,20.00,30.0,0.64,0.82,23.00,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,141994.0,152.0,56414.0,-1.73,0.11,39.73,39.88,259.0,0.00,1.05,43.9,XY24,NTT,POWER
60,NATIONALUM,189.63,247.44,-49.40,H-MC,70.83,101262.0,758.0,29882.0,0.42,0.75,29.51,30.49,97.0,0.03,0.75,33.4,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,9.38,17329.0,-32264.0,1187.0,7.67,-65.06,6.85,-62.66,NaN,-27.18,0.13,16.21,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,4.17,45686.0,4431.0,6373.0,7.56,10.74,13.95,26.19,1.0,0.70,0.34,7.56,XY24,ATH,IT
63,QUESS,296.83,986.00,-31.68,M-SC,31.77,68087.0,3081.0,147844.0,3.58,4.74,217.14,232.18,191.0,0.02,0.50,4.26,XY24,NTT,MISC
74,SIEMENS,4406.90,7969.85,-1.77,X-LC,35.42,83078.0,-27094.0,116168.0,2.20,-24.59,139.83,80.85,14.0,-0.23,0.61,22.36,AR,ATH,ELECTRICAL
33,GREENPANEL,375.16,537.00,202.97,M-SC,77.08,135249.0,-47829.0,126809.0,2.06,-26.12,93.76,43.14,216.0,-0.38,1.00,27.22,XY24,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,KPIGREEN,497.21,732.16,26.30,H-SC,64.58,129276.0,3979.0,55227.0,-3.00,3.18,42.72,47.25,139.0,0.07,0.95,61.73,MH,ATH,POWER
69,SAMMAANCAP,170.35,326.00,-171.59,M-SC,86.46,81828.0,-20382.0,113774.0,-2.87,-19.94,139.04,91.37,206.0,-0.18,0.60,32.83,XY25,NTT,FINANCE
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68371.0,-45178.0,85231.0,-2.65,-39.79,124.66,35.27,261.0,-0.53,0.50,67.34,XR,NTT,HOTELS
47,INDUSINDBK,1354.50,1800.00,79.87,L-MC,68.75,48801.0,-28406.0,53798.0,-2.61,-36.79,110.24,32.89,253.0,-0.53,0.36,34.39,XR,NTT,BANKS
32,ENRIN,1377.95,2975.90,NaN,NaN,71.88,72500.0,38051.0,1900.0,-2.41,110.46,2.62,115.97,NaN,20.03,0.53,10.86,AR,ATH,ELECTRICAL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,92.71,134136.0,-8343.0,8960.0,0.93,-5.86,6.68,0.44,243.0,-0.93,0.99,58.32,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-213.57,H-SC,97.92,81958.0,-13358.0,13400.0,-1.25,-14.01,16.35,0.04,154.0,-1.00,0.60,61.65,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-32.22,M-SC,5.21,144506.0,-26635.0,26575.0,-1.38,-15.56,18.39,-0.03,189.0,-1.00,1.06,3.86,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,124.33,H-SC,89.58,146952.0,-27607.0,27642.0,0.39,-15.82,18.81,0.02,127.0,-1.00,1.08,27.43,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.82,X-MC,16.67,86233.0,-16419.0,16419.0,-0.79,-15.99,19.04,-0.00,58.0,-1.00,0.64,31.08,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,4.17,45686.0,4431.0,6373.0,7.56,10.74,13.95,26.19,1.0,0.70,0.34,7.56,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,41.67,267376.0,-28558.0,122458.0,-0.05,-9.65,45.80,31.73,1.0,-0.23,1.97,7.02,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,58.33,287617.0,26016.0,119620.0,-0.00,9.94,41.59,55.67,2.0,0.22,2.12,16.66,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.84,X-LC,21.88,257354.0,-12993.0,47610.0,0.10,-4.81,18.50,12.81,6.0,-0.27,1.90,7.63,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-12.09,X-LC,29.17,278671.0,13245.0,43668.0,-1.17,4.99,15.67,21.44,9.0,0.30,2.05,11.50,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,36.46,173154.0,32314.0,12173.0,-1.52,22.94,7.03,31.59,89.0,2.65,1.28,50.41,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,71.88,72500.0,38051.0,1900.0,-2.41,110.46,2.62,115.97,NaN,20.03,0.53,10.86,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,94.40,M-MC,60.42,233469.0,8507.0,159156.0,1.68,3.78,68.17,74.53,185.0,0.05,1.72,35.55,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,26.30,H-SC,64.58,129276.0,3979.0,55227.0,-3.00,3.18,42.72,47.25,139.0,0.07,0.95,61.73,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.43,H-SC,53.12,118289.0,16285.0,28744.0,-0.44,15.97,24.30,44.14,124.0,0.57,0.87,34.80,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,36.46,173154.0,32314.0,12173.0,-1.52,22.94,7.03,31.59,89.0,2.65,1.28,50.41,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,71.88,72500.0,38051.0,1900.0,-2.41,110.46,2.62,115.97,NaN,20.03,0.53,10.86,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.18,M-SC,93.75,204021.0,32400.0,129676.0,0.62,18.88,63.56,94.43,198.0,0.25,1.50,50.58,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-4.22,M-LC,76.04,164269.0,13324.0,96130.0,0.15,8.83,58.52,72.51,69.0,0.14,1.21,13.15,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-48.08,H-SC,88.54,135076.0,11740.0,123095.0,-1.89,9.52,91.13,109.32,122.0,0.10,0.99,33.45,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,26.30,H-SC,64.58,129276.0,3979.0,55227.0,-3.00,3.18,42.72,47.25,139.0,0.07,0.95,61.73,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,9.38,17329.0,-32264.0,1187.0,7.67,-65.06,6.85,-62.66,NaN,-27.18,0.13,16.21,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68371.0,-45178.0,85231.0,-2.65,-39.79,124.66,35.27,261.0,-0.53,0.50,67.34,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-19.15,H-SC,62.50,91860.0,-6120.0,28853.0,-1.10,-6.25,31.41,23.21,137.0,-0.21,0.68,33.23,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,7.77,X-SC,98.96,100315.0,-648.0,95179.0,1.31,-0.64,94.88,93.63,64.0,-0.01,0.74,49.42,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.21,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,141994.0,152.0,56414.0,-1.73,0.11,39.73,39.88,259.0,0.00,1.05,43.90,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,7.77,X-SC,98.96,100315.0,-648.0,95179.0,1.31,-0.64,94.88,93.63,64.0,-0.01,0.74,49.42,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-213.57,H-SC,97.92,81958.0,-13358.0,13400.0,-1.25,-14.01,16.35,0.04,154.0,-1.00,0.60,61.65,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68371.0,-45178.0,85231.0,-2.65,-39.79,124.66,35.27,261.0,-0.53,0.50,67.34,XR,NTT,HOTELS
51,JIOFIN,310.58,387.00,6.28,H-LC,95.83,227815.0,10409.0,43080.0,-1.26,4.79,18.91,24.61,95.0,0.24,1.68,61.98,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.9
1,25,42.0
2,50,70.5


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.45
LC    34.05
MC    21.51
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.59
X40      12.25
XY25     12.02
X40N     11.80
XR       10.14
AR        8.22
OX40N     8.18
MH        1.70
X5K       1.48
X200      1.40
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.61
M-SC    15.99
H-LC    14.14
X-LC    12.20
H-MC     9.50
X-MC     7.54
M-LC     6.66
X-SC     4.39
M-MC     4.11
L-SC     2.46
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.12,-1.97,36.09
IT,12.12,-10.54,64.82
BANKS,7.98,-5.29,48.25
FINANCE,7.88,-12.70,53.49
MISC,6.65,-15.07,68.46
PAINTS,5.70,-12.89,35.14
ELECTRICAL,5.46,0.69,45.70
HEALTHCARE,4.53,-4.65,35.22
AUTO,4.33,-14.96,65.52


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2713397.0,27
XR,1084181.0,13
AR,1027565.0,10
X40,656045.0,9
X40N,559380.0,11
XY25,487719.0,8
OX40N,319273.0,11
SR,177248.0,2
MH,85109.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1984842.0,20
M-SC,1770505.0,21
H-LC,620249.0,12
H-MC,543306.0,8
X-LC,519001.0,7
X-MC,430456.0,6
M-MC,412320.0,3
X-SC,323335.0,5
M-LC,269178.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      867567.0      6
M-SC       XY24      816738.0      7
H-SC       AR        524023.0      3
M-SC       XR        334765.0      4
M-MC       XY24      303011.0      2
H-MC       XY24      300872.0      3
X-LC       X40       242078.0      2
H-SC       X40N      227153.0      4
           XR        218196.0      3
H-LC       X40N      201680.0      4
M-SC       AR        182628.0      2
           OX40N     168245.0      5
M-LC       XY24      159050.0      3
H-LC       X40       157700.0      3
X-MC       X40       153891.0      2
L-SC       XR        147671.0      2
X-SC       XR        124312.0      1
X-LC       XY25      120806.0      3
           AR        116168.0      1
M-SC       XY25      113774.0      1
M-MC       XR        109309.0      1
X-MC       XY25      104275.0      1
           XY24       97194.0      1
M-LC       XR         96130.0      1
X-SC       SR         95179.0      1
H-SC       OX40N      92676.0      3
H-MC       XY25       88139.0      1
M-SC       SR         82069.0      1
H-LC       AR         80492.0      1
M-SC       X40        72286.0      1
H-LC       X200       70159.0      1
           X5K        67138.0      2
H-MC       AR         62402.0      1
X-SC       XY24       61911.0      1
L-SC       AR         59952.0      1
X-MC       X40N       58677.0      1
L-SC       XY24       56414.0      1
H-SC       MH         55227.0      1
L-MC       XR         53798.0      1
L-LC       XY25       46727.0      1
H-LC       XY24       43080.0      1
X-SC       OX40N      41933.0      2
X-LC       X40N       39949.0      1
H-MC       X40N       31921.0      1
           X40        30090.0      1
           MH         29882.0      1
X-MC       OX40N      16419.0      1
M-LC       XY25       13998.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
